# Capítulo 8: redução de dimensionalidade

In [1]:
%matplotlib qt 

Um problema recorrente no ML é a quantidade obscena de features que precisamos lidar em alguns datasets.

Isso não só torna o treino lento como pode dificultar a busca de boas soluções.

Além disso, redução de dimensionalidade é extremamente útil para visualização!

Apelidamos o problema de 'maldição da dimensionalidade' (*curse of dimensionality*)

exemplo: 

para identificar um número no mnist podemos: 

* descartar as bordas brancas
* diminuir a resolução da imagem

e, ainda assim, ter informação suficiente para determinar de qual dígito se trata.

## The Curse of Dimensionality

Algumas coisas se comportam de forma diferente em espaços de alta dimensionalidade.

Escolha um ponto aleatório dentro de um quadrado 1x1. ele terá 0.4% de chance de estar a menos de 0.001 de uma borda.

No entanto, em um hipercubo com 10k dimensões, a probabilidade disso ocorrer sobe para 99.999999%

No código abaixo a gente compara entre 1k dimensões vs 2 dimensões

In [2]:
import numpy as np
from numpy import random as rnd

d1k = rnd.uniform(size = (1000, 100000))
d2 = rnd.uniform(size = (2, 100000))

print('chance para 10k dimensões: ', sum(sum(np.logical_or(d1k<0.001, d1k> 1- 0.001)) != 0)/100000)

print('chance para 2 dimensões: ', sum(sum(np.logical_or(d2<0.001, d2> 1- 0.001)) != 0)/100000)

chance para 10k dimensões:  0.86473
chance para 2 dimensões:  0.00386


Em um cubo unitário 2d (quadrado), a distância média entre 2 pontos aleatórios é de 0.52 

Para o cubo 3d: 0.66

Para o cubo 1000000d: 408.25


O fato é: datasets de alta dimensionalidade correm o risco de serem esparsos!

As instâncias de treino estarão distantes entre si e, provavelmente, novas instâncias surgirão longe de todos os dados de treino.

Isso tem um impacto forte sobre a confiabilidade das previsões feitas. Como podemos prever instâncias distantes das instâncias que conhecíamos?

Em resumo: maior dimensionalidade $\implies$ mais chance de overfittar

Na teoria podemos aumentar o número de instâncias para aumentar a densidade de dados. Na prática o número de dados necessários para manter a densidade acompanhando a dimensão cresce exponencialmente com a dimensão :/.

Com 100 features, você precisa de mais dados de treino do que o número de átomos no universo observável para conseguir instâncias espaçadas por 0.1 entre elas na média. (assumindo dispersão uniforme)

## Principais abordagens para redução de dimensionalidade

### Projeção

As vezes é possível acomodar a distribuição dos dados em espaços menores que o espaço original. exemplo:

![](./imgs/subspace.png)

![](./imgs/subspace2.png)

No entanto, nem sempre é possível projetar de forma legal...

![](./imgs/proj.png)

Queremos uma forma de 'desenrolar' esse rolo.

### Manifold Learning

Esse rolo acima é um exemplo de um manifold 2d. Um manifold é um formato 2d que pode ser dobrado e espremido em um espaço dimensional maior. Como um pano de cozinha.

Em modo mais geral é um objeto d-dimensional forçadamente inserido em um espaço n-dimensional com $n \gt d$.

Os 'constraints implícitos' de um conjunto de dados 'obriga' os dados de alta dimensão a serem acomodáveis em uma dimensão menor que a que eles pertencem. 

No MNIST, por exemplo, os números são escritos com linhas contínuas, isso implica que os pixels não possuem tantos graus de liberdade quanto poderiam. Logo eles devem se acomodar em um espaço dimensional menor.

Usar manifold vem da crença que o dataset vai ser mais facilmente explicável em dimensões menores, o que nem sempre é verdade. Vejamos na imagem:

![](./imgs/manifold.png)

## PCA

Principals Component Analysis (PCA) é de longe o algoritmo de redução de dimensionalidade mais famoso.

Primeiro ele identifica o hiperplanos que melhor acomoda o dataset e, em seguida, projeta os dados nele.

![](./imgs/pca1.png)

Parece justificável escolher o eixo que preserva a maior variância. Outra forma de justificar a escolha é que esse eixo minimiza a distância quadrática dos dados e suas projeções no eixo.

Essa é a ideia por trás do PCA

### Principal components

O PCA acha n eixos (vetores) ortogonais entre si para projetar os dados. o i-ésimo vetor é chamado de i-ésimo componente principal (PC).

A técnica usada para achar os PC's é chamada de SVD (singular value decomposition). É basicamente achar os auto-vetores e auto-valores da matriz.

**Importante:** PCA assume que os dados estão centralizados. Portanto, centralize seus dados antes de usá-lo.

### Como reduzir a dimensão?

Descarta os últimos componentes principais. Isso fará com que você preserve o máximo de variância possível ao eliminar features.

In [39]:
from sklearn.decomposition import PCA
from sklearn.datasets import make_s_curve

X, y = make_s_curve(5000)
pca = PCA(n_components = 2)
X2d = pca.fit_transform(X, y)

In [40]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax3d = Axes3D(fig)
ax3d.scatter(X[:,0], X[:,1], X[:,2], c = y)
plt.show()

In [41]:
plt.figure()
plt.scatter(X2d[:,0], X2d[:,1], c = y )
plt.show()

### Explained Variance Ratio

Indica a proporção do dataset que se acomoda ao longo do eixo de cada componente principal.

In [47]:
evr = pca.explained_variance_ratio_

print('evr por componente: ',evr,'\n\nVariância total preservada: ', sum(evr))

evr por componente:  [0.69909465 0.17868298] 

Variância total preservada:  0.8777776350977695


### Escolhendo o número correto de dimensões

Regra de bolso: número que cubra 95% da variância. (usar explained_variance_ratio)

pca = PCA(n_components = 0.95)

Outra opção é plotar o explained_variance_ratio vs número de componentes e encontrar no plot um número interessante de componentes.

![](./imgs/pca2.png)

perceba que isso é mais um trade-off!

## PCA para compressão

Reduzir a dimensão é, também, comprimir os dados. Usando PCA para o MNIST com 95% de variância preservada reduz de 784 features para 150!

jogamos fora mais de 80% do dataset, acelerando pakas o processo de treino.

tem como 'descomprimir' para as 784 dimensões e visualizar o que foi perdido.

Resultado:

![](./imgs/pca3.png)

### Mais PCA

#### Incremental

É possível treinar em mini-batches. Bom para rodar online e para datasets grandes.

#### Randomized

Estocástico. Encontra um valor aproximado para os primeiros d componentes principais.

#### Kernel

Lembram do kernel trick do SVM?

O mesmo truque serve pro PCA, possibilitando fazer projeções não lineares complexas para a redução de dimensionalidade.

In [159]:
from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_friedman3

X, y = make_swiss_roll(5000, random_state=0)
pca = KernelPCA(n_components = 2, kernel = 'sigmoid', gamma = 1E-3)
X2d = pca.fit_transform(X, y)

In [160]:
plt.figure()
plt.scatter(X2d[:,0], X2d[:,1], c = y )
plt.show()

### Selecionando um kernel e tunando hiperparâmetros

grid_search.

KernelPCA é unsupervised mas vc pode usar a previsão do modelo seguinte da pipeline para verificar quais são os melhores parâmetros para o KernelPCA.

Uma abordagem completamente unsupervised é selecionar o kernel e hiperparâmetros que acusam o menor erro de reconstrução - reconstruir não é tão trivial nesse caso. 

Página 219 para mais informações sobre isso.

## LLE - Locally Linear Embedding

Técnica de manifold learning que não se baseia em projeção. Esse modelo mede como cada instância de treino se relaciona linearmente com seu vizinho mais próximo.

Então procura por uma representação de menor dimensão onde essas relações locais são melhores preservadas.

É especialmente boa para desenrolar datasets enrolados, especialmente quando se tem pouco ruído.

In [190]:
from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

X, y = make_swiss_roll(5000, random_state=0)


lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X2d = lle.fit_transform(X)

In [191]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax3d = Axes3D(fig)
ax3d.scatter(X[:,0], X[:,1], X[:,2], c = y)
plt.show()

In [192]:
plt.figure()
plt.scatter(X2d[:,0], X2d[:,1], c = y )
plt.show()

O algorítmo identifica os K vizinhos mais próximos de $\pmb{x}^{(i)}$ e tenta reconstruir $\pmb{x}^{(i)}$ como uma função linear desses vizinhos.

Encontra pesos que multiplicam os k vizinhos para tentar igualar a $\pmb{x}^{(i)}$.

Criamos um vetor para cada instância. Ou seja, criamos uma matriz. Essa matriz codifica as relações locais lineares entre as instâncias de treino.

$$
\hat{\pmb{W}} =  \underset{W}{\mathrm{argmin}}\sum_{i=1}^{m}||\pmb{x}^{(i)} - \sum_{j = 1}^{m} w_{i,j}\pmb x^{(j)}||^2
$$

Mas $w_{i,j} = 0$ para quando $x^{(j)}$ não é um dos vizinhos de $x^{(i)}$.

O próximo passo é mapear as instâncias em um espaço d-dimensional ($d \lt n$) preservando essas relações lineares.

Criamos uma instância $\pmb{z}^{(i)}$ para cada $\pmb{x}^{(i)}$ em um novo espaço d-dimensional (z é a imagem de x nesse novo espaço). Com o vetor de pesos fixo tentamos descrever $\pmb{z}^{(i)}$ através de seus vizinhos.

$$
\hat{\pmb{Z}} =  \underset{Z}{\mathrm{argmin}}\sum_{i=1}^{m}||\pmb{Z}^{(i)} - \sum_{j = 1}^{m} w_{i,j}\pmb Z^{(j)}||^2
$$

Basicamente ele tenta achar um lugar legal para colocar a imagem de x nesse novo espaço.

## Outras técnicas

* Multidimensional Scaling (MDS) reduces dimensionality while trying to preserve
the distances between the instances

* Isomap creates a graph by connecting each instance to its nearest neighbors, then
reduces dimensionality while trying to preserve the geodesic distances between
the instances.

* t-Distributed Stochastic Neighbor Embedding (t-SNE) reduces dimensionality
while trying to keep similar instances close and dissimilar instances apart. It is
mostly used for visualization, in particular to visualize clusters of instances in
high-dimensional space

* Linear Discriminant Analysis (LDA) is actually a classification algorithm, but dur‐
ing training it learns the most discriminative axes between the classes, and these
axes can then be used to define a hyperplane onto which to project the data. The
benefit is that the projection will keep classes as far apart as possible, so LDA is a
good technique to reduce dimensionality before running another classification
algorithm such as an SVM classifier.
